In [1]:
import matplotlib.pyplot as plt
import  numpy as np
import tensorflow as tf
import pandas as pd
float_formatter = lambda x: "%.3f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})


In [124]:
#Interface Scale
#ΔGwif (kcal/mol) 	Octanol Scale
#ΔGwoct (kcal/mol) 	Octanol − Interface
#Scale
dictt = {'A': 20, 'C': 1, 'E': 2, 'D': 3, 'G': 4,
         'F': 5, 'I': 6, 'H': 7, 'K': 8, 'M': 9,
         'L': 10, 'N': 11, 'Q': 12, 'P': 13, 'S': 14,
         'R': 15, 'T': 16, 'W': 17, 'V': 18, 'Y': 19, 'X' : 0  } #switched ala to 20 and 'X'
dictt_hydropathy  = {
'I' : [ 	-0.31, 	-1.12, 	-0.81 ,0 ],
'L' : [ 	-0.56,	-1.25, 	-0.69 ,0],
'F' : [ 	-1.13, 	-1.71,	-0.58 ,0],
'V' : [ 	0.07 ,	-0.46, 	-0.53 ,0],
'M' : [ 	-0.23, 	-0.67,	-0.44 ,0],
'P' : [ 	0.45 ,	0.14 ,	-0.31 ,0],
'W' : [ 	-1.85, 	-2.09, 	-0.24 ,0],
'T' : [ 	0.14 , 	0.25 , 	0.11 ,0],
'Q' : [ 	0.58 ,	0.77 ,	0.19 ,0],
'C' : [ 	-0.24,	-0.02, 	0.22 ,0],
'Y' : [ 	-0.94, 	-0.71, 	0.23 ,0],
'A' : [ 	0.17 ,	0.50, 	0.33 ,0],
'S' : [ 	0.13 ,	0.46, 	0.33 ,0],
'N' : [ 	0.42 ,	0.85, 	0.43 ,0],
'R' : [ 	0.81 ,	1.81, 	1.00 ,1],
'G' : [ 	0.01 ,	1.15, 	1.14 ,0],
'H' : [ 	0.96 ,	2.33, 	1.37 ,0],
'E' : [ 	2.02 ,	3.63, 	1.61 ,-1],
'K' : [ 	0.99 ,	2.80, 	1.81 ,1],
'D' : [ 	1.23 ,	3.64, 	2.41 ,-1],
'Z' : [ 1.3,  2.2,  0.9, -0.5], #E or Q (mass spec data cannot differentiate)
'B' : [ 0.825,  2.245,  1.42 , -0.5  ], #D or N (mass spec data cannot differentiate)
'X' : [0.,0.,0.,0.]} 


In [6]:
data = pd.read_csv('bioactive_PDB_cpp.csv').dropna()

In [7]:
def fn1(str):
    num=0.0
    for i in str:
        if i.upper() == 'R' or i.upper() == 'K':
            num += 1
    if num/len(str) > .334: #should be tweaked
        return 0
    return len(set(str))
def fn2(str): # find if sequence has weird bonds
    for i in list(set(str)):
        if i.upper() == '-':
            return 2
    for i in list(set(str)):
        if i.upper() not in dictt:
            return 1    
    return 0


In [125]:
data['type_aa'] = data['seq'].apply(fn1)
data = data[data['type_aa'] >= 4]
print 'removing these number of non-cannonical peptides' ,len(data[data['seq'].apply(fn2) == 1])
data = data[data['seq'].apply(fn2) == 0] # remove peptides with weird chemical bonds , and non-cannonical res (mostly negavtives)
data['len'] = data.seq.apply(len)
data = data.sort_values(by = ['len','source']).reset_index(drop=True)
# get frequencies of amino acids
all = np.concatenate([data.seq])
result = {}
for i in all:
    for j in i :
        if j not in result:
            result[j] =1
        else:
            result[j] += 1
print result
total_aa = np.sum([result[x] for x in result])
for i in result:
    result[i] = np.round(result[i]*1.0 /total_aa,3)
print result


removing these number of non-cannonical peptides 0
{'A': 13809, 'C': 6853, 'E': 6730, 'D': 6110, 'G': 13341, 'F': 7237, 'I': 6698, 'H': 3514, 'K': 8574, 'M': 5073, 'L': 21706, 'N': 4807, 'Q': 5262, 'P': 9896, 'S': 11657, 'R': 9044, 'T': 7066, 'W': 3261, 'V': 9026, 'Y': 4492}
{'A': 0.084000000000000005, 'C': 0.042000000000000003, 'E': 0.041000000000000002, 'D': 0.036999999999999998, 'G': 0.081000000000000003, 'F': 0.043999999999999997, 'I': 0.041000000000000002, 'H': 0.021000000000000001, 'K': 0.051999999999999998, 'M': 0.031, 'L': 0.13200000000000001, 'N': 0.029000000000000001, 'Q': 0.032000000000000001, 'P': 0.059999999999999998, 'S': 0.070999999999999994, 'R': 0.055, 'T': 0.042999999999999997, 'W': 0.02, 'V': 0.055, 'Y': 0.027}


In [159]:
X = []
counter = 0
## creates num of residue...
features = []
for i in dictt:
    if i != 'X':
        def fn(str):
            result = 0.0
            for j in str:
                if j.upper() == i:
                    result += 1
            return result
        data['num_'+str(i)] = data['seq'].apply(fn)
        data['per_'+str(i)] = data['num_'+str(i)]/data['len']
        features += ['per_'+str(i),'num_'+str(i)]

for idx in range(len(data)):
    i = data.iloc[idx]['seq'].upper()
    #i = i + (30-len(i))*'X'
    temp = np.zeros((len(i),5))
    for j in  range(len(i)):
        res = dictt[i[j]]
        temp[j][0] = res
        #temp[j][-1] = len(i)*0.01
        temp[j][-4:] = dictt_hydropathy[i[j]]
    temp = temp.T
    alternative = [len(i),np.sum(temp[-1,:]),np.sum(temp[-2,:]),np.sum(temp[-3,:])]
    alternative += list(data.iloc[idx][features]) #add ratio for aa
    per = data.iloc[idx]['source'] == 2 
    X += [[temp[0],temp[1:],alternative,i,per*1],]
    #print '> %s\n%s' %(i,i)
    counter += 1
data['X'] = X


In [130]:
X[-1000]

[array([4.000, 10.000, 5.000, 10.000, 3.000, 16.000, 10.000, 8.000, 4.000,
        10.000, 20.000, 4.000, 8.000, 10.000, 10.000, 12.000, 4.000, 10.000,
        8.000, 1.000, 6.000, 8.000, 20.000, 4.000, 1.000, 8.000, 13.000,
        0.000, 0.000, 0.000]),
 array([[0.010, -0.560, -1.130, -0.560, 1.230, 0.140, -0.560, 0.990, 0.010,
         -0.560, 0.170, 0.010, 0.990, -0.560, -0.560, 0.580, 0.010, -0.560,
         0.990, -0.240, -0.310, 0.990, 0.170, 0.010, -0.240, 0.990, 0.450,
         0.000, 0.000, 0.000],
        [1.150, -1.250, -1.710, -1.250, 3.640, 0.250, -1.250, 2.800, 1.150,
         -1.250, 0.500, 1.150, 2.800, -1.250, -1.250, 0.770, 1.150, -1.250,
         2.800, -0.020, -1.120, 2.800, 0.500, 1.150, -0.020, 2.800, 0.140,
         0.000, 0.000, 0.000],
        [1.140, -0.690, -0.580, -0.690, 2.410, 0.110, -0.690, 1.810, 1.140,
         -0.690, 0.330, 1.140, 1.810, -0.690, -0.690, 0.190, 1.140, -0.690,
         1.810, 0.220, -0.810, 1.810, 0.330, 1.140, 0.220, 1.810, -0.310,
  

In [109]:
import tensorflow.contrib.keras as keras
from tensorflow.contrib.keras.api.keras.layers import Input, Embedding, GRU, Dense,concatenate,Masking ,Dropout,  Lambda#, Activation
from tensorflow.contrib.keras.api.keras.models import Model
from tensorflow.contrib.keras.python.keras.layers import Bidirectional, TimeDistributed
from tensorflow.contrib.keras.api.keras.backend import permute_dimensions, concatenate
from tensorflow.contrib.keras.api.keras.callbacks import ModelCheckpoint


In [12]:
batch_size = None
epsilon = 1e-3
def batch_normalization(x,name='batchnorm',feature_norm = False):
    # ideally i want to do batch norm per row per sample
    #epsilon = tf.Variable(tf.constant([1e-3,]*x.shape[0])) 
    if feature_norm : 
##        mean,var = tf.nn.moments(x,[2,3],keep_dims=True)
##        scale = tf.Variable(tf.ones([1,x.shape[1],1,1]))
##        beta = tf.Variable(tf.zeros([1,x.shape[1],1,1]))
##        x = tf.nn.batch_normalization(x,mean,var,beta,scale,epsilon,name=name)
        x = tf.contrib.layers.layer_norm (x,trainable=False)
    else:
        x = tf.contrib.layers.layer_norm (x,trainable=False)
##        mean,var = tf.nn.moments(x,[1,2],keep_dims=True)
##        scale = tf.Variable(tf.ones([1,1,1,x.shape[-1]]))
##        beta = tf.Variable(tf.zeros([1,1,1,x.shape[-1]]))
##        x = tf.nn.batch_normalization(x,mean,var,beta,scale,epsilon,name=name)
    return x

In [158]:

#Inp0 = tf.placeholder(tf.int32,[batch_size,None],name='sequence_factors1')
#Inp1 = tf.placeholder(tf.float32,[batch_size,4,None],name='sequence_factors2')
#labels = tf.placeholder(tf.float32 , [batch_size,1],name='labels')
#Inp2 = tf.placeholder(tf.float32, [batch_size,44] ,name = 'globa_seq_info')
Inp0 = Input(shape=(None,), dtype='int32', name='sequence_factors1')
Inp1 = Input(shape=(4,None,), dtype='float32', name='sequence_factors2') # this is batch_size, 4, None to be same with cnn
Inp1_reshape  = Lambda( lambda x : permute_dimensions(x,(0,2,1)))(Inp1)
Inp2 = Input(shape=(44,), dtype='float32', name='globa_seq_info')
embedded_sequences = Embedding(20,
                                5,
                                input_length=30,
                                trainable=True, 
                                mask_zero=False,
                                name="SentencesEmbedded") (Inp0)
#layer0 = concatenate([Inp1_reshape,embedded_sequences])
layer0 = Lambda( lambda x : concatenate([x[0],x[1]]),name='concat_seqe') ([Inp1_reshape,embedded_sequences])
#unstack0 = tf.unstack(Inp1,axis=-2,name='unstack0')
#unstack1 = tf.unstack(embedded_sequences , axis=-2,name='unstack1')
#layer0 = tf.stack([tf.stack(unstack0+unstack1,axis=1)],-1,name='stack')

ValueError: Tensor conversion requested dtype int32 for Tensor with dtype float32: 'Tensor("maskinginp0_3/masking/Cast_1:0", shape=(?, 1), dtype=float32)'

In [102]:
BATCH_SIZE = 32
RNN_HIDDEN_SIZE = 300
print layer0

Tensor("concat_RNNinput/concat:0", shape=(?, ?, 9), dtype=float32)


In [103]:

rnn_outputs = Bidirectional( GRU(RNN_HIDDEN_SIZE, return_sequences=False),  merge_mode='concat' )(layer0)
rnn_outputs2 = Dropout(0.5,name='drop')(rnn_outputs)
gbmp_extra =  Lambda( lambda x : concatenate([x[0],x[1]],axis = 1), name ='gbmp_extra') ([Inp2,rnn_outputs2])
is_ner_outputs = Dense(256, activation='relu',name='dense1')(gbmp_extra)
is_ner_outputs = Dense(128, activation='relu',name='dense2')(is_ner_outputs)
is_ner_outputs = Dense(64, activation=None,name='dense3')(is_ner_outputs)
is_ner_outputs = Dense(1, activation='sigmoid',name='out')(is_ner_outputs)


In [142]:
model = Model(inputs=[Inp0,Inp1,Inp2], outputs=[is_ner_outputs])
model.compile(loss='binary_crossentropy', optimizer='adam')
#model = Model(inputs=[Inp0,Inp1,Inp2], outputs=[layer0])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
sequence_factors2 (InputLayer)   (None, 4, None)       0                                            
____________________________________________________________________________________________________
sequence_factors1 (InputLayer)   (None, None)          0                                            
____________________________________________________________________________________________________
lambda_15 (Lambda)               (None, None, 4)       0           sequence_factors2[0][0]          
____________________________________________________________________________________________________
SentencesEmbedded (Embedding)    (None, None, 5)       100         sequence_factors1[0][0]          
___________________________________________________________________________________________

In [143]:
filepath="./weights-{epoch:02d}-{loss:.4f}-gentext-CharRNN.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [161]:
def get_batch_from_X(Xy): #get batch tensor inputs from X and y
    Inp0_ = np.array([x[0] for x in Xy])
    Inp1_ = np.array([x[1] for x in Xy])
    Inp2_ = np.array([x[2] for x in Xy])
    labels_ = np.array([[x[-1] for x in Xy],])
    return Inp0_,Inp1_,Inp2_,labels_.T
Inp0_,Inp1_,Inp2_,labels_ = get_batch_from_X(X)

ValueError: could not broadcast input array from shape (4,4) into shape (4)

In [154]:
print Inp2_[0:5]

[[30.000 0.000 1.380 1.730 0.000 0.000 0.250 1.000 0.000 0.000 0.000 0.000
  0.250 1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
  0.000 0.000 0.000 0.000 0.000 0.000 0.250 1.000 0.250 1.000 0.000 0.000
  0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [30.000 0.000 0.630 1.290 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
  0.250 1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
  0.000 0.000 0.000 0.000 0.000 0.000 0.250 1.000 0.250 1.000 0.000 0.000
  0.000 0.000 0.000 0.000 0.250 1.000 0.000 0.000]
 [30.000 0.000 4.240 7.080 0.250 1.000 0.000 0.000 0.000 0.000 0.250 1.000
  0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.250 1.000 0.000 0.000
  0.000 0.000 0.000 0.000 0.000 0.000 0.250 1.000 0.000 0.000 0.000 0.000
  0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [30.000 0.000 0.890 0.650 0.250 1.000 0.000 0.000 0.000 0.000 0.000 0.000
  0.250 1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
  0.250 1.000 0.000 0.000 0.0

In [160]:
model.fit([Inp0_,Inp1_,Inp2_],labels_, epochs=1, batch_size=1, callbacks=callbacks_list)

Epoch 1/1


InvalidArgumentError: indices[0,4] = 20 is not in [0, 20)
	 [[Node: SentencesEmbedded_39/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](SentencesEmbedded_19/embeddings/read, _arg_sequence_factors1_21_0_4)]]

Caused by op u'SentencesEmbedded_39/Gather', defined at:
  File "/home/leexa/anaconda2/envs/tensorflow/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/leexa/anaconda2/envs/tensorflow/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/leexa/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/leexa/anaconda2/envs/tensorflow/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/leexa/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/kerne